In [4]:
    @staticmethod
    def pydantic_to_gemini_schema(schema: Any) -> Dict[str, Any]:
        """
        Convert a Pydantic schema to Gemini API schema format.
        
        Args:
            schema: Pydantic schema to convert
            
        Returns:
            Schema in Gemini API format
        """
        if hasattr(schema, "model_json_schema"):
            json_schema = schema.model_json_schema()
            
            # Convert JSON schema to Gemini API format
            # This is a simplified implementation that should be expanded
            # based on the specific Gemini API requirements
            gemini_schema = {
                "type": "OBJECT",
                "properties": {}
            }
            
            for prop_name, prop_info in json_schema.get("properties", {}).items():
                prop_type = prop_info.get("type")
                
                if prop_type == "string":
                    gemini_schema["properties"][prop_name] = {"type": "STRING"}
                elif prop_type == "integer":
                    gemini_schema["properties"][prop_name] = {"type": "INTEGER"}
                elif prop_type == "number":
                    gemini_schema["properties"][prop_name] = {"type": "NUMBER"}
                elif prop_type == "boolean":
                    gemini_schema["properties"][prop_name] = {"type": "BOOLEAN"}
                elif prop_type == "array":
                    gemini_schema["properties"][prop_name] = {"type": "ARRAY"}
                elif prop_type == "object":
                    gemini_schema["properties"][prop_name] = {"type": "OBJECT"}
            
            # Add required fields
            if "required" in json_schema:
                gemini_schema["required"] = json_schema["required"]
                
            return gemini_schema
        
        # If not a Pydantic model, return as is (assuming it's already in correct format)
        return schema

    @staticmethod
    def python_to_gemini_function(
        function: Callable,
        name: Optional[str] = None,
        description: Optional[str] = None
    ) -> Dict[str, Any]:
        """
        Convert a Python function to a Gemini API function definition.
        
        Args:
            function: Function to convert
            name: Optional custom name for the function
            description: Optional custom description
            
        Returns:
            Function definition in Gemini API format
        """
        # Get function metadata
        func_name = name or function.__name__
        func_description = description or inspect.getdoc(function) or ""
        
        # Get signature and type hints
        sig = inspect.signature(function)
        type_hints = get_type_hints(function)
        
        # Create parameters schema
        parameters = {
            "type": "OBJECT",
            "properties": {},
            "required": []
        }
        
        # Process parameters
        for param_name, param in sig.parameters.items():
            # Skip self for methods
            if param_name == "self":
                continue
                
            # Get parameter type
            param_type = type_hints.get(param_name, str)
            param_schema = ConversationManagementUtils._python_type_to_schema(param_type)
            
            # Add to properties
            parameters["properties"][param_name] = param_schema
            
            # If no default value, it's required
            if param.default == inspect.Parameter.empty:
                parameters["required"].append(param_name)
        
        # Create function definition
        function_def = {
            "name": func_name,
            "description": func_description,
            "parameters": parameters
        }
        
        # Add return type if available
        if "return" in type_hints:
            return_type = type_hints["return"]
            if return_type is not None:
                function_def["response"] = ConversationManagementUtils._python_type_to_schema(return_type)
        
        return function_def
    
    @staticmethod
    def _python_type_to_schema(python_type: Any) -> Dict[str, Any]:
        """
        Convert Python type to Gemini schema type.
        
        Args:
            python_type: Python type annotation
            
        Returns:
            Schema type in Gemini API format
        """
        # Handle primitive types
        if python_type == str:
            return {"type": "STRING"}
        elif python_type == int:
            return {"type": "INTEGER"}
        elif python_type == float:
            return {"type": "NUMBER"}
        elif python_type == bool:
            return {"type": "BOOLEAN"}
        
        # Handle Union/Optional types
        origin = get_origin(python_type)
        if origin is Union:
            args = get_args(python_type)
            # Handle Optional (Union with None)
            if type(None) in args:
                non_none_args = [arg for arg in args if arg is not type(None)]
                if len(non_none_args) == 1:
                    base_type = ConversationManagementUtils._python_type_to_schema(non_none_args[0])
                    base_type["nullable"] = True
                    return base_type
            
            # General Union as anyOf
            schemas = [ConversationManagementUtils._python_type_to_schema(arg) for arg in args 
                      if arg is not type(None)]
            return {"anyOf": schemas}
        
        # Handle List
        if origin is list or python_type == List:
            args = get_args(python_type)
            item_type = str  # Default item type
            if args:
                item_type = args[0]
            return {
                "type": "ARRAY",
                "items": ConversationManagementUtils._python_type_to_schema(item_type)
            }
        
        # Handle Dict
        if origin is dict or python_type == Dict:
            return {"type": "OBJECT"}
        
        # Default to string for unknown types
        return {"type": "STRING"}
    
    @staticmethod
    def prepare_functions(functions: List[Callable]) -> List[Dict[str, Any]]:
        """
        Convert a list of Python functions to Gemini API format.
        
        Args:
            functions: List of functions to convert
            
        Returns:
            List of function declarations in API format
        """
        return [ConversationManagementUtils.python_to_gemini_function(func) for func in functions]
    
    @staticmethod
    def extract_function_calls(response: Any) -> List[Dict[str, Any]]:
        """
        Extract function calls from a model response.
        
        Args:
            response: Model response
            
        Returns:
            List of function calls
        """
        # Parse response if it's a string
        if isinstance(response, str):
            try:
                response_data = json.loads(response)
            except json.JSONDecodeError:
                # Not JSON, no function calls
                return []
        elif isinstance(response, dict):
            response_data = response
        else:
            return []
        
        # Check for function calls in different formats
        function_calls = []
        
        # Format 1: functionCalls array
        if "functionCalls" in response_data:
            return response_data["functionCalls"]
            
        # Format 2: function_calls array
        if "function_calls" in response_data:
            return response_data["function_calls"]
            
        # Format 3: functionCall in candidates
        if "candidates" in response_data:
            for candidate in response_data["candidates"]:
                if "functionCall" in candidate:
                    function_calls.append(candidate["functionCall"])
                    
        # Format 4: toolCalls in various schemas
        tool_calls = response_data.get("toolCalls", []) or response_data.get("tool_calls", [])
        if tool_calls:
            for tool in tool_calls:
                if isinstance(tool, dict) and "function" in tool:
                    function_info = tool["function"]
                    # Convert to standard format
                    function_call = {
                        "name": function_info.get("name"),
                        "args": function_info.get("arguments", {})
                    }
                    
                    # Parse args if it's a string
                    if isinstance(function_call["args"], str):
                        try:
                            function_call["args"] = json.loads(function_call["args"])
                        except json.JSONDecodeError:
                            function_call["args"] = {}
                            
                    function_calls.append(function_call)
        
        return function_calls